## Main Authors:
* Fabbrico Davide
* Khateeb Ahmad Arsalan
* Pagnoncelli Diego

In [31]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import plotly.express as px
import plotly.io as pio

pd.set_option('display.max_rows', 1000)
pd.set_option('mode.chained_assignment', None)
pio.templates.default = "ggplot2"

In [32]:
# read total dataframe
df_tot = pd.read_csv('total_dataframe.csv').drop('Unnamed: 0', axis=1)
df_tot['NT'] = 1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning:

Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.



In [33]:
def PlotGreenGraph(G):
    labels = {e: str(e) for e in G.edges}
    # pos = nx.spring_layout(G, k=10000/np.sqrt(G.order()), iterations=20, scale=10) problem divided by 0
    pos = nx.spring_layout(G, iterations=20, scale=10)
    plt.figure(3, figsize=(20, 14))
    
    nx.draw(G, pos=pos, with_labels=True, node_size=2000, node_color='orange')

    # Create edge labels
    labels = {e: G.edges[e]['Quantity'] for e in G.edges}

    # Draw edge labels according to node positions
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    
    return None

In [34]:
def GreennessGraph(df, month = 1, year = 2019, type_flow = 'export', product = None, t = 1):
    
    # product
    if product != None or product != '':
        df = df[df['PRODUCT_CODE']==product].reset_index(drop=True)
    else:
        return 'Select the product code!'
    
    # month and year
    if month == None or year == None or month > 12 or year > 2021 or year < 2019:
        return 'Insert correctly the month and year!'
    
    if month != None and year != None:
        df = df[(df['MONTH'] == month) & (df['YEAR'] == year)].reset_index(drop=True)

    df = df[df['TYPE_FLOW'] == type_flow].reset_index(drop=True) 
    
    df['Distance'] = round(df['Distance'], 3)
    
    # start changes
    df = df.groupby(['DECLARANT_ISO', 'PARTNER_ISO', 'PRODUCT_CODE','TYPE_TRANSP_MODE', 'Distance'])[['DECLARANT_ISO', 'PARTNER_ISO', 'VALUE_IN_EUROS', 'QUANTITY_IN_KG', 'NT']].sum().reset_index()
    df = df.loc[df['TYPE_TRANSP_MODE'].isin(['Road', 'Air', 'Sea', 'Rail','Inland Waterway'])].reset_index(drop=True)
    speed = 45
    
    def greeness(row):
        if row['TYPE_TRANSP_MODE'] == 'Sea':
            return row['QUANTITY_IN_KG']/(row['NT']*(9936.70 * row['Distance'] / (speed*1000)))
        elif row['TYPE_TRANSP_MODE'] == 'Air':
            return row['QUANTITY_IN_KG']/(row['NT']*(12*row['Distance']/1000))
        elif row['TYPE_TRANSP_MODE'] == 'Inland Waterway':
            return row['QUANTITY_IN_KG']/(row['NT']*(0.12*row['Distance']/1000))
        elif row['TYPE_TRANSP_MODE'] == 'Road':
            return row['QUANTITY_IN_KG']/(row['NT']*(0.25*row['Distance']/1000))
        elif row['TYPE_TRANSP_MODE'] == 'Rail':
            return row['QUANTITY_IN_KG']/(row['NT']*(7.97*row['Distance']/1000))

    df['Greenness'] = df.apply(lambda row: greeness(row), axis=1) 
    
    tr = int(len(df)*t/100)
    df = df.iloc[:tr+1]
    
    G = nx.DiGraph()
    # nodes
    nodes = df['DECLARANT_ISO'].append(df['PARTNER_ISO']).unique()
    
    # graph
    G.add_nodes_from(nodes)

    for index,row in df.iterrows():
        G.add_edge(row['DECLARANT_ISO'],row['PARTNER_ISO'],
                                             Value = row.VALUE_IN_EUROS,
                                             Quantity = row.QUANTITY_IN_KG,
                                            Distance = row.Distance,
                                            Greenness = row.Greenness)

    
    return G

In [35]:
def DFGreenGraph(G, month=None, year=None):
    dict_summ = {}
    for i in G.nodes():
        summ = 0
        for j in G[i]:
            summ += G[i][j]['Greenness'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summ[i] = summ
        
    dict_summQ = {}
    for i in G.nodes():
        summQ = 0
        for j in G[i]:
            summQ += G[i][j]['Quantity'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summQ[i] = summQ

    df_greenG = pd.DataFrame(list(dict_summ.items()), columns=['Country', 'Greenness']).sort_values(by='Greenness', ascending=False).reset_index(drop=True)
    df_greenQ = pd.DataFrame(list(dict_summQ.items()), columns=['Country', 'Quantity'])
    df_GQ = df_greenG.merge(df_greenQ, on='Country')
    df_GQ['month'] = month
    df_GQ['year'] = year
    df_GQ['date'] = datetime.datetime(year, month, 1)
    df_GQ = df_GQ[df_GQ['Greenness']>0]
    return df_GQ

In [36]:
year = [2019, 2020, 2021]
month = [1,2,3,4,5,6,7,8,9,10,11,12]
df = pd.DataFrame({})
for y in year:
    if y == 2021:
        month = list(range(1, 7))   
    for m in month:
        greenG = GreennessGraph(df_tot, m, y, product = '210690', t = 10)
        greenDF = DFGreenGraph(greenG, month=m, year=y)
        df = pd.concat([df, greenDF], copy=False)
df = df.reset_index(drop=True)

In [40]:
df = df[df['Country']=='AT']
fig = px.line(df, x="month", y="Greenness", animation_frame="year", animation_group="Country", hover_name="Country", markers=True)
fig["layout"]
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    title="Index of Greenness (2019-2021)",
    xaxis_title="Month",
    yaxis_title="Greenness",
    title_x=0.5,
    font=dict(
        size=16,
    )
)
# fig.update_layout(title_text='Index of Greenness (2019-2021)', title_x=0.5)
fig.update_xaxes(tickmode = 'array',
                 tickvals = df['month'].unique(),
                 ticktext= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])